In [2]:
#import lib
import pandas as pd
import numpy as np

In [86]:
attr = pd.read_table('attr.txt', sep=":", usecols=all, names = ['attr', 'range'])
train = pd.read_table('train.txt', sep="\s+", usecols=all, names = list(attr['attr']))

#clean data:
train = train.drop_duplicates()
train = train.dropna()

#get test set
X_va = pd.read_table('sample-test.txt', sep="\s+", usecols=all, names = list(attr['attr'])[:-1])
y_va = pd.read_table('predicted.txt', usecols=all,  names = ['Class'])
train.shape

(11982, 177)

In [97]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [87]:
class FeatureSelection:
    def __init__(self, x_in, y_in):
        self.x = x_in
        self.y = y_in

    def model(self, model_name):
        # Choose model
        match model_name:
            case 'extra_tree':
                return FeatureSelection.extra_tree(self)
            case 'univariate':
                return FeatureSelection.univariate(self)
            case 'rand_forest':
                return FeatureSelection.rand_forest(self)

    def extra_tree(self):
        # Tree-based feature selection
        clf = ExtraTreesClassifier(n_estimators=100, criterion='entropy', random_state=42, max_depth=9)
        clf = clf.fit(self.x, self.y)
        ext_model = SelectFromModel(clf, prefit=True)
        x_new = ext_model.transform(self.x)
        print('ExtraTree feature:', x_new.shape)
        return x_new

    def univariate(self):
        # Univariate feature selection
        x_new = SelectKBest(chi2, k=5).fit_transform(self.x, self.y)
        print('Univariate feature:', x_new.shape)
        return x_new

    def rand_forest(self):
        clf = RandomForestClassifier(n_estimators=100, max_depth=9, random_state=42)
        clf.fit(self.x, self.y)
        ext_model = SelectFromModel(clf, prefit=True)
        x_new = ext_model.transform(self.x)
        print('ExtraTree feature:', x_new.shape)
        return x_new

    def dis_feature(self):

        return

    def linear_feature(self):
        return




In [112]:
class SplitModel:
    def __init__(self, x_in, y_in):
        self.x = x_in
        self.y = y_in

    def model(self, model_name):
        # Choose model
        match model_name:
            case 'rand_split':
                return SplitModel.rand_split(self)
            case 'distribution_plit':
                return SplitModel.distribution_split(self)


    def rand_split(self):
        return train_test_split(self.x, self.y , test_size=0.2, random_state = 42)

    def distribution_split(self):
        return


In [119]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

In [121]:
class TrainingModel:
    def __init__(self, x_tr, x_te, y_tr, y_te):
        self.x_tr = x_tr
        self.x_te = x_te
        self.y_tr = y_tr
        self.y_te = y_te

    def model(self, model_name):
        # Choose model
        match model_name:
            case 'gaussianNB':
                return TrainingModel.gaussian_NB(self)
            case 'SVM':
                return TrainingModel.SVM(self)

    def gaussian_NB(self):
        gnb = GaussianNB()
        gnb.fit(self.x_tr, self.y_tr)
        # Check accuracy score
        y_NB_pred = gnb.predict(self.x_te)
        return accuracy_score(self.y_te, y_NB_pred)

    def SVM(self):
        clf = svm.SVC()
        clf.fit(self.x_tr, self.y_tr)
        y_SVM_pred = clf.predict(self.x_te)
        return accuracy_score(self.y_te, y_SVM_pred)


In [126]:
x_train = train[train.columns[:-1]]
y_train = train['Class']

# Feature selection
feature_model = ['extra_tree', 'univariate', 'rand_forest']
fea_se = FeatureSelection(x_train, y_train)
x_new = fea_se.model(feature_model[0])

# Split data
split_way = ['rand_split', 'distribution_plit']
split = SplitModel(x_new, y_train)
X_train, X_test, y_train, y_test = split.model(split_way[0])

#Training
training_method = ['gaussianNB', 'SVM']
training_model = TrainingModel(X_train, X_test, y_train, y_test)
print('Accuracy:',training_model.model(training_method[0]))

ExtraTree feature: (11982, 11)
Accuracy: 0.7129745515227367


D:\Project\python\venv\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
